In [1]:
import requests
import json
import pandas as pd
import numpy as np
from flatten_json import flatten

import pandas as pd

# Read the file with the NPI data
input_prompt = input("Enter the raw file path: ")
df = pd.read_csv(input_prompt)

# Write NPIs into a list - 'NPI number': col name for NPI
npi_list = df['NPI number'].tolist()
# Convert the NPI list to string
npi_list = [str(i) for i in npi_list]
# API string from NPPES
api = 'https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&name_purpose=&first_name=&use_first_name_alias=&last_name=&organization_name=&address_purpose=&city=&state=&postal_code=&country_code=&limit=&skip=&pretty=&version=2.1'
# Create lists to save to
code_list = []
taxonomy_list = []
result_list = []
primary_list = []
# the index in the API string where NPI insert happens
insert_index = 44

# Loop thru NPI list
for npi in npi_list:
    # Use current NPI to make API string
    new_api = api[:insert_index] + npi + api[insert_index:]
    # pull request
    r = requests.get(new_api)
    # save result as json
    result_json = r.json()
    # check if the [taxonomies] stub in result has more than 1 values, if yes, go into the loop, if no, get [code] and [desc] from the first value
    if len(result_json['results'][0]['taxonomies']) > 1:
        # loop through the values
        for i in range (len(result_json['results'][0]['taxonomies'])):
            # under current [taxonomies] value, get [primary] value
            primary = result_json['results'][0]['taxonomies'][i]['primary']
            # if [primary] value is 'True', get the [code] and [desc] values, else pass
            if primary:
                code = result_json['results'][0]['taxonomies'][i]['code']
                taxonomy = result_json['results'][0]['taxonomies'][i]['desc']
                code_list.append(code)
                taxonomy_list.append(taxonomy)
                primary_list.append(primary)
            else:
                pass
            
    else: 
        primary = result_json['results'][0]['taxonomies'][0]['primary']
        code = result_json['results'][0]['taxonomies'][0]['code']
        taxonomy = result_json['results'][0]['taxonomies'][0]['desc']
        code_list.append(code)
        taxonomy_list.append(taxonomy)
        primary_list.append(primary)

# Make a new dataframe
new_data = {'NPI': npi_list, 'Provider taxonomy': code_list, 'Specialty': taxonomy_list, 'Primary': primary_list}
new_df = pd.DataFrame(data=new_data)

# Write frame to csv
output_prompt = input("Enter the output dir: ")
output_filename = 'NPIlookup_result.csv'
new_df.to_csv(output_prompt + output_filename)




KeyboardInterrupt: Interrupted by user